In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import tensorflow as tf



In [ ]:
# Step 1: Load the spectral reflectance and soil property data
def load_data(spectral_path, soil_properties_path):
    spectral_data = pd.read_csv(spectral_path)
    soil_data = pd.read_csv(soil_properties_path)
    # Merge datasets on the ID column
    merged_data = pd.merge(spectral_data, soil_data, on='ID', how='inner')
    return merged_data


In [ ]:

# Step 2: Apply PCA to select informative bands
def apply_pca(data, n_components):
    pca = PCA(n_components=n_components)
    transformed_data = pca.fit_transform(data)
    return transformed_data, pca



In [ ]:
# Step 3: Train the ANN model
def train_ann(X_train, y_train):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer with 1 neuron for regression
    ])
    opt = Adam(learning_rate== 0.028)
    model.compile(optimizer=opt, loss='mse')
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
    return model




In [ ]:
# Step 4: Generate the equation from the trained ANN model
def generate_equation(model):
    weights = model.get_weights()
    equation = ""
    for layer_idx, (w, b) in enumerate(zip(weights[0::2], weights[1::2])):
        equation += f"Layer {layer_idx + 1}: Weights = {w.shape}, Biases = {b.shape}\n"
    return equation

# Step 5: Validate the generated equation on both LUCAS-2009 and AFSIS dataset



In [ ]:
# Main script
if __name__ == "__main__":
    # File paths (update these paths to your file locations)
    spectral_path = "spc.csv"
    soil_properties_path = "soil_data.csv"

    # Load and preprocess data
    data = load_data(spectral_path, soil_properties_path)
    spectral_columns = [col for col in data.columns if col.startswith('Band')]
    X = data[spectral_columns]
    y = data['Target']  # Replace 'Target' with the appropriate column name

    # Apply PCA to reduce spectral dimensions
    X_pca, pca_model = apply_pca(X, n_components=n_components)
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

    # Train ANN model
    ann_model = train_ann(X_train, y_train)

    # Generate the equation
    equation = generate_equation(ann_model)
    print("Generated Equation:\n", equation)

    # Validate the equation using various evaluation metrics R2, RMSE, MAE and RPD